# Get data from OSRM

The script is to request route data from OSRM http://project-osrm.org/. There are three variables will be retrieved for each trip in train data or test data: the fastest route trip duration, the fastest route trip distance and a column containing route step coordinates. We can modify the urls to request more variables from OSRM such as distance per step, I just include three variables as a start-up. 

In [ ]:
import urllib.request
import requests
import json
import pandas as pd
import multiprocessing
import time

taxi=pd.read_csv("train.csv")
#taxi=pd.read_csv("test.csv")

taxi['pickup_location']=taxi['pickup_longitude'].astype(str)+','+taxi['pickup_latitude'].astype(str)
taxi['dropoff_location']=taxi['dropoff_longitude'].astype(str)+','+taxi['dropoff_latitude'].astype(str)
taxi['url']="http://router.project-osrm.org/route/v1/driving/"+taxi['pickup_location']+";"+taxi['dropoff_location']+"?geometries=geojson&overview=simplified&steps=false"
taxi['index1'] = taxi.index
taxi.shape


In [3]:
#taxi1=taxi[0:500000]
#url1=taxi1[['index1','id','url']].values.tolist()
#urls=[ tuple(i) for i in url1]
#print(len(urls))
#print(taxi1.shape)

def get_direction(idx,idee,url):
        
                json_list=[]
                
                headers = {'Pengchong Tang': 'my-app for kaggle.com competition'}
                
                while True:
                    response = requests.get(url,headers=headers)
                    direction=response.content.decode('utf-8')

                    try:
                        var=json.loads(direction)
                        if var!={'message': 'Too Many Requests'} :
                                distance=var['routes'][0]['distance']
                                duration=var['routes'][0]['duration']
                                route_coordinates=var['routes'][0]['geometry']['coordinates'] 
                                json_list.append({'id': idee,'duration': duration,'distance':distance,'route_coordinates': route_coordinates})
                                
                                break
                    except :
                        json_list.append({'id': idee })
                        break

        
                
                if idx % 1000==0:
                      print(str(idx)+" done")

                return json_list

In [6]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

filenum=0
for taxi1 in chunker(taxi,60000):
    if filenum >=1:
        url1=taxi1[['index1','id','url']].values.tolist()
        urls=[ tuple(i) for i in url1]

        pool = multiprocessing.Pool(100)  # Number of CPUs

        result=pool.starmap(get_direction,urls)
        pool.close()

        result1=[ x[0] for x in result]
        direction_train=pd.DataFrame(result1)
        filename="direction"+str(filenum)+".csv"
        direction_train.to_csv(filename,index=False)
    
    filenum+=1

0 done
3000 done
6000 done
9000 done
12000 done
2000 done
5000 done
8000 done
14000 done
11000 done
10000 done
1000 done
4000 done
7000 done
13000 done
15000 done
18000 done
21000 done
24000 done
27000 done
17000 done
20000 done
23000 done
26000 done
29000 done
16000 done
19000 done
22000 done
25000 done
28000 done
30000 done
33000 done
39000 done
42000 done
36000 done
32000 done
35000 done
41000 done
44000 done
38000 done
31000 done
34000 done
45000 done
37000 done
43000 done
40000 done
48000 done
51000 done
47000 done
50000 done
54000 done
46000 done
57000 done
49000 done
53000 done
52000 done
56000 done
59000 done
55000 done
58000 done
